In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
import random

# Set seeds for reproducbility across different model tests
SEED = 40
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Need this for cross-compatibility as we trained and tested on Google Colab GPU
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

# Preprocessing
# Finding mean and std to normalise dataset
temp_transform = transforms.ToTensor()
temp_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=temp_transform)
temp_loader = DataLoader(temp_trainset, batch_size=128, shuffle=False, num_workers=2)

mean = torch.zeros(3)
std = torch.zeros(3)
no_of_batches = 0

for x, y in temp_loader:
    mean += x.mean(dim=[0,2,3])
    std += x.std(dim=[0,2,3])
    no_of_batches += 1

mean /= no_of_batches
std /= no_of_batches
print("Mean: ", mean)
print("Std: ", std)


In [10]:
# Allows transformation without having to import training dataset twice and manually splitting for training or validation
class CIFAR10DatasetTransform(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        image, label = self.dataset[index]
        if self.transform:
            image = self.transform(image)
        return image, label

#### Transform and Load the Datasets

In [ ]:
# Regular transformation for validation and test sets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean.tolist(), std.tolist())
])

""" Improvement from Base CNN: Data augmentation transformation - only for training set """
transform_trainset = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean.tolist(), std.tolist())
])

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=None)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 9:1 ratio split for training:validation from entire training set
training_set_size = 45000
validation_set_size = 5000

train_subset, validation_subset = random_split(cifar_dataset, [training_set_size, validation_set_size])
training_dataset = CIFAR10DatasetTransform(train_subset, transform_trainset)
validation_dataset = CIFAR10DatasetTransform(validation_subset, transform)

# Num_workers depends on system cores
train_loader = DataLoader(training_dataset, batch_size=128, shuffle=True, num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(cifar_testset, batch_size=128, shuffle=False, num_workers=2)

#### CNN Class

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Starting dimensions 32 x 32 x 64 -> 16 x 16 x 128
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv1_3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bnorm1_1 = nn.BatchNorm2d(64)
        self.bnorm1_2 = nn.BatchNorm2d(64)
        self.bnorm1_3 = nn.BatchNorm2d(64)
        
        # Starting dimensions 16 x 16 x 128 -> 8 x 8 x 128
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3 ,stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv2_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bnorm2_1 = nn.BatchNorm2d(128)
        self.bnorm2_2 = nn.BatchNorm2d(128)
        self.bnorm2_3 = nn.BatchNorm2d(128)


        # Starting dimensions 8 x 8 x 256 -> 4 x 4 x 256
        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bnorm3_1 = nn.BatchNorm2d(256)
        self.bnorm3_2 = nn.BatchNorm2d(256)
        self.bnorm3_3 = nn.BatchNorm2d(256)

        # Reduces H X W dimensions by half (32 x 32 -> 16 x 16)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Activation function
        self.relu = nn.ReLU()

        """ Improvement from Base CNN Model: Dropout - Enables better generalisation
            Dropout before fully-connected-layer 
            Dropout between convolutions to generalise better """
        self.dropoutfcl = nn.Dropout(p=0.4)
        self.dropoutconv = nn.Dropout2d(p=0.1)
        
        # Final fully connected layer, input (4 x 4 x 256) dimensions should be flattened
        self.fully_connected_layer = nn.Linear(4*4*256, 10)
        
        # Cross entropy loss
        self.cross_entropy_loss = nn.CrossEntropyLoss()


    def forward(self, input):
        # Conv Block 1
        input = self.relu(self.bnorm1_1(self.conv1_1(input)))
        input = self.relu(self.bnorm1_2(self.conv1_2(input)))
        input = self.dropoutconv(self.relu(self.bnorm1_3(self.conv1_3(input))))
        input = self.maxpool(input)

        # Conv Block 2
        input = self.relu(self.bnorm2_1(self.conv2_1(input)))
        input = self.relu(self.bnorm2_2(self.conv2_2(input)))
        input = self.dropoutconv(self.relu(self.bnorm2_3(self.conv2_3(input))))
        input = self.maxpool(input)

        # Conv Block 3
        input = self.relu(self.bnorm3_1(self.conv3_1(input)))
        input = self.relu(self.bnorm3_2(self.conv3_2(input)))
        input = self.dropoutconv(self.relu(self.bnorm3_3(self.conv3_3(input))))
        input = self.maxpool(input)
        
        # Flatten + FCL
        input = self.fully_connected_layer(self.dropoutfcl(torch.flatten(input, 1, -1)))

        return input

#### Training Loop

In [ ]:
import torch.optim as optim

cnn_model = CNN().to(device)
optimiser = optim.Adam(cnn_model.parameters(), lr=0.001)
""" Improvement from Base CNN: Learning Rate Scheduler """
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode='min',factor=0.5, patience=3, min_lr=0.00001)

# Logging metrics
accuracy_history = []
loss_history = []
validation_loss_history = []
validation_accuracy_history = []

epochs = 100

# Main training loop
for i in range(epochs):
    # Set to train mode
    cnn_model.train()
    total_loss = 0
    training_correct = 0
    for x_train, y_train in train_loader:
        x_train, y_train = x_train.to(device), y_train.to(device)
        optimiser.zero_grad()
        output = cnn_model(x_train)
        loss = cnn_model.cross_entropy_loss(output, y_train)
        loss.backward()
        optimiser.step()
        total_loss += loss.item()
        training_correct += (torch.argmax(output, dim=1) == y_train).sum().item()
        
    
    avg_loss = total_loss / len(train_loader)
    training_accuracy = training_correct / len(train_loader.dataset)
    loss_history.append(avg_loss)
    accuracy_history.append(training_accuracy)

    # Testing on the validation set
    cnn_model.eval()
    validation_loss = 0
    validation_correct = 0

    with torch.no_grad():
        for x_validate, y_validate in validation_loader:
            x_validate, y_validate = x_validate.to(device), y_validate.to(device)
            output = cnn_model(x_validate)
            
            validation_loss += cnn_model.cross_entropy_loss(output, y_validate).item()
            validation_correct += (torch.argmax(output, dim=1) == y_validate).sum().item()
    
    avg_validation_loss = validation_loss / len(validation_loader)
    validation_accuracy = validation_correct / len(validation_loader.dataset)

    validation_loss_history.append(avg_validation_loss)
    validation_accuracy_history.append(validation_accuracy)
    print(f"Epoch {i} Loss: {avg_loss:.4f}, Accuracy: {training_accuracy:.4f}, Validation Loss: {avg_validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}")
    for param_group in optimiser.param_groups:
        lr = param_group['lr']
    print(f"Learning Rate: {lr}")

    scheduler.step(avg_validation_loss)

#### Testing Loop

In [ ]:
total_correct = 0
total_samples = 0
test_loss = 0
# Set model to eval mode 
cnn_model.eval()
with torch.no_grad():
    # Looping over the test set in batches
    for x_test, y_test in test_loader:
        x_test, y_test = x_test.to(device), y_test.to(device)
        output = cnn_model(x_test)
        loss = cnn_model.cross_entropy_loss(output, y_test)
        test_loss += loss.item()
        y_pred = torch.argmax(output, dim=1)
        correct_pred = torch.sum(y_pred == y_test)
        batch_accuracy = correct_pred.item() / len(x_test)
        total_correct += correct_pred.item()
        total_samples += len(x_test)
accuracy = total_correct / total_samples
avg_test_loss = test_loss / len(test_loader)
print("Correct predictions: ", total_correct)
print(f"Accuracy: {accuracy:.2f}, Test Loss: {avg_test_loss:.4f}")

#### Saving model logs for analysis

In [ ]:
# Turn all logs into NumPy arrays for logging into .npz file
training_loss_log = np.array(loss_history)
training_accuracy_log = np.array(accuracy_history)
validation_loss_log = np.array(validation_loss_history)
validation_accuracy_log = np.array(validation_accuracy_history)
test_loss_log = np.array(avg_test_loss)
test_accuracy_log = np.array(accuracy)

In [ ]:
import os
# Versioning for the experiment should be as descriptive as possible 
# and include any tweaks made as part of training/testing that version of the model
experimental_model_type = "cnn_base_v1_"
filename = f"cnn-logs/{experimental_model_type}.npz"
if os.path.exists(filename):
    raise FileExistsError("File name already exists use another name")
os.makedirs("cnn-logs", exist_ok=True)
np.savez(filename, training_loss_log, training_accuracy_log, validation_loss_log, validation_accuracy_log, test_loss_log, test_accuracy_log)

##### Save and load the best performing model

In [ ]:
# Only do this for best performing model
import pickle
os.makedirs("saved-models", exist_ok=True)
with open("saved-models/cnn_model.pkl", "wb") as f:
    pickle.dump(cnn_model, f)

In [ ]:
with open("saved-models/cnn_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)